# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [2]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [3]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [4]:
data = pd.read_csv("data/kddcup.data_10_percent", header=None, names = col_names)

# 前処理
## カテゴリ化

In [5]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [6]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [7]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [8]:
data['label3'] = data.label.copy()

In [9]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop'),'label3'] = 'DoS'

In [10]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit'),'label3'] = 'U2R'

In [11]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster'),'label3'] = 'R2L'

In [12]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan'),'label3'] = 'Probe'

In [13]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

## サンプリング

In [14]:
data = resample(data,n_samples=15000,random_state=0)

In [15]:
data.shape

(15000, 44)

## 数値化

In [16]:
le_protocol_type = preprocessing.LabelEncoder()

In [17]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [18]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [19]:
le_service = preprocessing.LabelEncoder()

In [20]:
le_service.fit(data.service)

LabelEncoder()

In [21]:
data.service = le_service.transform(data.service)

In [22]:
le_flag = preprocessing.LabelEncoder()

In [23]:
le_flag.fit(data.flag)

LabelEncoder()

In [24]:
data.flag = le_flag.transform(data.flag)

In [25]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,15000.000000,15000.000000,15000.000000,15000.000000,1.500000e+04,15000.000000,15000.0,15000.000000,15000.0,15000.000000,...,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,43.627333,0.461600,22.213933,5.300733,1.586459e+03,582.709933,0.0,0.007933,0.0,0.039800,...,232.891533,189.160000,0.755863,0.030312,0.605959,0.007220,0.174164,0.173948,0.060811,0.059823
std,596.907820,0.575164,13.411106,1.514435,5.939465e+04,6497.232653,0.0,0.150572,0.0,0.855725,...,64.078727,105.863893,0.410113,0.107856,0.480673,0.047621,0.378417,0.378777,0.235377,0.234500
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.0,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,13.000000,6.000000,4.400000e+01,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,53.750000,0.450000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,13.000000,6.000000,5.200000e+02,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,39.000000,6.000000,1.032000e+03,0.000000,0.0,0.000000,0.0,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,21661.000000,2.000000,61.000000,7.000000,5.133876e+06,646195.000000,0.0,3.000000,0.0,30.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
data.shape

(15000, 44)

## ラベルの分離

In [27]:
y_train_1 = data.label.copy() 

In [28]:
y_train_2 = data.label2.copy()

In [29]:
y_train_3 = data.label3.copy()

In [30]:
x_train = data.drop(['label','label2','label3'],axis=1)

In [31]:
x_train.shape

(15000, 41)

In [32]:
y_train_1.shape

(15000,)

In [33]:
y_train_2.shape

(15000,)

In [34]:
y_train_3.shape

(15000,)

## 標準化

In [35]:
ss = preprocessing.StandardScaler()

In [36]:
ss.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [37]:
x_train = ss.transform(x_train)

In [38]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [39]:
pd.DataFrame(x_train,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,15000.0,1.500000e+04,15000.0,1.500000e+04,...,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04,1.500000e+04
mean,-4.981367e-16,2.115870e-15,-4.090358e-16,1.942668e-16,9.944415e-16,4.661382e-16,0.0,-1.250371e-15,0.0,-2.127290e-15,...,-2.191765e-15,2.423543e-16,1.139844e-15,6.570374e-16,-3.042011e-16,2.504663e-17,-6.631806e-16,-3.327634e-16,7.659577e-16,-7.918111e-16
std,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,0.0,1.000033e+00,0.0,1.000033e+00,...,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00,1.000033e+00
min,-7.309133e-02,-8.025798e-01,-1.656439e+00,-3.500257e+00,-2.671137e-02,-8.968886e-02,0.0,-5.268975e-02,0.0,-4.651184e-02,...,-3.618974e+00,-1.777436e+00,-1.843121e+00,-2.810520e-01,-1.260688e+00,-1.516203e-01,-4.602595e-01,-4.592506e-01,-2.583663e-01,-2.551183e-01
25%,-7.309133e-02,-8.025798e-01,-6.870605e-01,4.617498e-01,-2.597053e-02,-8.968886e-02,0.0,-5.268975e-02,0.0,-4.651184e-02,...,3.450319e-01,-1.279138e+00,-7.458267e-01,-2.810520e-01,-1.260688e+00,-1.516203e-01,-4.602595e-01,-4.592506e-01,-2.583663e-01,-2.551183e-01
50%,-7.309133e-02,-8.025798e-01,-6.870605e-01,4.617498e-01,-1.795608e-02,-8.968886e-02,0.0,-5.268975e-02,0.0,-4.651184e-02,...,3.450319e-01,6.219514e-01,5.953105e-01,-2.810520e-01,8.197971e-01,-1.516203e-01,-4.602595e-01,-4.592506e-01,-2.583663e-01,-2.551183e-01
75%,-7.309133e-02,9.361113e-01,1.251696e+00,4.617498e-01,-9.335486e-03,-8.968886e-02,0.0,-5.268975e-02,0.0,-4.651184e-02,...,3.450319e-01,6.219514e-01,5.953105e-01,8.982687e-02,8.197971e-01,-1.516203e-01,-4.602595e-01,-4.592506e-01,-2.583663e-01,-2.551183e-01
max,3.621680e+01,2.674802e+00,2.892182e+00,1.122084e+00,8.641284e+01,9.937059e+01,0.0,1.987201e+01,0.0,3.501266e+01,...,3.450319e-01,6.219514e-01,5.953105e-01,8.990920e+00,8.197971e-01,2.084842e+01,2.182419e+00,2.180910e+00,3.990287e+00,4.009409e+00


## 学習

In [40]:
pca = PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [41]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [42]:
x_train2 = pca.transform(x_train)

In [43]:
x_train2.shape

(15000, 3)

In [44]:
clf = SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1.0, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
t1=time.perf_counter()
clf.fit(x_train2,y_train_2)
t2=time.perf_counter()

In [46]:
print(t2-t1,"秒")

1332.138791933001 秒


## 予測

In [47]:
t1=time.perf_counter()
pred=clf.predict(x_train2)
t2=time.perf_counter()


In [48]:
print(t2-t1,"秒")

0.09199034299672348 秒


In [49]:
print(classification_report(y_train_2, pred))
print(confusion_matrix(y_train_2, pred))

             precision    recall  f1-score   support

      atack       1.00      0.99      0.99     12140
    normal.       0.94      0.99      0.97      2860

avg / total       0.99      0.99      0.99     15000

[[11969   171]
 [   30  2830]]


## 学習

In [50]:
pca2 = PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [51]:
clf2 = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.17782794100389229,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [52]:
pca2.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [53]:
x_train3 = pca2.transform(x_train)

In [54]:
t1=time.perf_counter()
clf2.fit(x_train3,y_train_3)
t2=time.perf_counter()

In [55]:
print(t2-t1,"秒")

0.21833463800430764 秒


## 予測

In [56]:
t1=time.perf_counter()
pred2=clf2.predict(x_train3)
t2=time.perf_counter()


In [57]:
print(t2-t1,"秒")

0.13323847600258887 秒


In [58]:
print(classification_report(y_train_3, pred2))
print(confusion_matrix(y_train_3, pred2))

             precision    recall  f1-score   support

        DoS       1.00      0.99      0.99     11960
      Probe       0.99      0.80      0.88       138
        R2L       1.00      0.15      0.27        39
        U2R       1.00      0.67      0.80         3
    normal.       0.95      1.00      0.97      2860

avg / total       0.99      0.99      0.99     15000

[[11840     1     0     0   119]
 [   19   110     0     0     9]
 [    0     0     6     0    33]
 [    0     0     0     2     1]
 [    1     0     0     0  2859]]
